# Euristica per la risoluzione del problema di  Pompei-Orienteering

In [1]:
class Edge:
    def __init__(self, inode, jnode, cost):
        """
        Initialise.
        :param inode: The starting node.
        :param jnode: The ending node.
        :param cost: The length of the path from inode to jnode.
        """
        self.inode = inode
        self.jnode = jnode
        self.cost = cost
        self.savings = dict()

In [2]:
import collections


class Node:
    def __init__(self, id, x, y, revenue, *, color='#FDDD71', issource=False, vehicles=0, isdepot=False):
        """
        Initialise.

        :param id: The unique id of the node.
        :param x: The x-coordinateof the node.
        :param y: The y-coordinate of the node.
        :param revenue: The revenue.
        :param issource: A boolean variable that says if the node is a source.
        :param isdepot: A boolean variable that says if the node is the depot.
        :param vehicles: The number of vehicles starting from this node (it is 0
                         if the node is not a source).

                    *** Parameters used by the Mapper ***
        :attr assigned: True if the node is assigned to a source and 0 otherwise
        :attr preferences: Used in case of source node for the round-robin process.
        :attr nodes: Used in case of source for keeping the nodes assigned to it.

                    *** Parameters used by the PJS ***
        :attr from_source: The length of the current path from the source to this node.
        :attr to_depot: The length of the current path from this node to the depot.
        :attr route: The current route corresponding to the node.
        :attr link_left: True if the node is linked to the source, False otherwise.
        :attr link_right: True if the node is linked to the depot, False otherwise.
        """
        self.id = id
        self.x = x
        self.y = y
        self.revenue = revenue
        self.issource = issource
        self.vehicles = vehicles
        self.isdepot = isdepot

        # Attributes used by the Mapper
        self.assigned = False
        self.preferences = collections.deque()
        self.nodes = collections.deque()

        # Attributes used by the PJS
        self.from_source = 0
        self.to_depot = 0
        self.route = None
        self.link_left = False
        self.link_right = False


    def __copy__(self):
        obj = Node.__new__(self.__class__)
        obj.__dict__.update(self.__dict__)
        return obj


    def __repr__(self):
        return f"Node {self.id}"


    def __hash__(self):
        return self.id

In [3]:
import random
import math


def greedy (preferences):
    """
    This is a greedy iterator of preferences.
    It iterates the source preferences from the best to the worst,
    and as soon as it meets a node which has not been assigned yet,
    it returns it.

    :param preferences: The iterable list of preferences.
    """
    for _, node in preferences:
        if not node.assigned:
            yield node




def BRA (preferences, beta=0.3):
    """
    This method carry out a biased-randomised selection over the list of preferences.
    The selection is based on a quasi-geometric function:
                    f(x) = (1 - beta) ^ x
    and it therefore prioritise the first elements in list.

    :param preferences: The set of options already sorted from the best to the worst.
    :param beta: The parameter of the quasi-geometric distribution.
    :return: The element picked at each iteration.
    """
    L = len(preferences)
    options = list(preferences)
    for _ in range(L):
        idx = int(math.log(random.random(), 1.0 - beta)) % len(options)
        _, node = options.pop(idx)
        if not node.assigned:
            yield node

In [4]:
import numpy as np
import itertools
import operator


def _reset_assignment (node):
    """
    Method used to reset the affiliation of a node to a source.
    """
    node.assigned = False
    return node


def mapper (problem, iterator):
    """
    An instance of this class represents the Mapper.
    The Mapper is engaged to assign each node to visit to a source.

    NOTE: This method change the problem nodes in place.

    :param problem: The instance of the Multi-Source Team Orienteering Problem to solve.
    :param iterator: The iterator used by source to pick the preferences.
    :return: A mapping --i.e., a 2-dimensional np.array where element (i, j) is 1 if
            node j is assigned to source i, and 0 otherwise.
    """
    # Extract the characteristics of the problem
    dists = problem.dists
    sources, nodes, depot = problem.sources, problem.nodes, problem.depot
    n_sources, n_nodes = len(problem.sources), len(problem.nodes)

    # Reset the source the nodes belongs to
    nodes = tuple(map(_reset_assignment, nodes))

    # Compute the absolute distances
    abs_dists = np.array([[dists[s.id, n.id] for n in nodes] for s in sources]).astype("float32")
    # NOTE: Suggestion to present
    #abs_dists = np.array([[dists[s.id, n.id] + dists[n.id, depot.id] for n in nodes] for s in sources]).astype("float32")

    # Compute the marginal distances
    for i, source in enumerate(sources):
        marginal_dists = abs_dists[i,:] - np.concatenate((abs_dists[:i,:], abs_dists[i:,:],), axis=0).min(axis=0)
        source.preferences = iterator(sorted(zip(marginal_dists, nodes), key=operator.itemgetter(0)))
        source.nodes = collections.deque()


    # Assign nodes to sources
    # Init the number of nodes already assigned and the mapping matrix
    n_assigned = 0
    mapping = np.zeros((n_sources, n_sources + n_nodes))
    _null_element = object()
    # NOTE: Until nodes are not concluded a source at each turn pick a number of preferred
    # nodes that depend on the number of vehicles it has.
    for source in itertools.islice(itertools.cycle(sources), n_nodes):
        # Consider the preferences of the currently considered source
        preferences = source.preferences
        # Pick a number of preferences that depend on the number of vehicles
        # that start from the source.
        for _ in range(source.vehicles):
            # Pick a node
            picked_node = next(preferences, _null_element)
            # If the generator is exhausted exit the loop
            if picked_node is _null_element:
                break
            # Assign the node to the source
            source.nodes.append(picked_node)
            picked_node.assigned = True
            mapping[source.id, picked_node.id] = 1
            n_assigned += 1

        # If all the nodes have already been assigned we exit the loop
        if n_assigned == n_nodes:
            break

    # Return the mapping
    return mapping

In [5]:
import functools
import heapq


class Route:
    """
    An instance of this class represents a Route --i.e., a path
    from the source to the depot made by a vehicle.
    """
    def __init__(self, source, depot, starting_node):
        """
        Initialise.
        :param source: The source of the route.
        :param depot: The depot of the route.
        :param starting_node: The first node included into the route.

        :attr nodes: The nodes part of the route.
        :attr revenue: The total revenue of the route.
        :attr cost: The total cost of the route.
        """
        self.source = source
        self.depot = depot
        self.nodes = collections.deque([starting_node])
        self.revenue = starting_node.revenue
        self.cost = starting_node.from_source + starting_node.to_depot

    def merge (self, other, edge, dists):
        """
        This method merges in place this route with another.

        :param other: The other route.
        :param edge: The edge used for merging.
        :param dists: The matrix of distances between nodes.
        """
        # Get the cost and the nodes of the edge
        inode, jnode = edge.inode, edge.jnode
        edge_cost = dists[inode.id, jnode.id]
        # Update the list of nodes in the route
        self.nodes.extend(other.nodes)
        # inode is not conneted to the depot anymore
        # and jnode is not connected to the source anymore
        inode.link_right = False
        jnode.link_left = False
        # Update the revenue and the cost of this route
        self.cost += edge_cost - inode.to_depot + (other.cost - jnode.from_source)
        self.revenue += other.revenue
        # Update the route the nodes belong to
        for node in other.nodes:
            node.route = self


def _bra (edges, beta):
    """
    Biased randomised selection of the edges.

    :param edges: The list of edges.
    :param beta: The parameter of the quasi-geometric distribution.
    :return: The retrieved edge.
    """
    L = len(edges)
    options = list(edges)
    for _ in range(L):
        idx = int(math.log(random.random(), 1.0 - beta)) % len(options)
        yield options.pop(idx)


def PJS (problem, source, nodes, depot, beta):
    """
    An implementation of the Panadero Juan Savings heuristic algorithm.
    It is generally used to solve a single source team orienteering problem.

    :param problem: The instance of the problem to solve.
    :param source: The source for which the PJS will be used.
    :param nodes: The customers nodes to visit.
    :param depot: The destination depot.
    :param beta: The parameter of the biased randomisation (i.e. close to 1 for a greedy behaviour)

    :return: The routes the vehicles starting from source will make.
    """
    # Move useful references to the stack
    n_vehicles, nodes = source.vehicles, set(nodes),
    dists, Tmax = problem.dists, problem.Tmax
    source_id, depot_id = source.id, depot.id

    # Filter edges keeping only those that interest this subset of nodes and sort them
    sorted_edges = sorted([e for e in problem.edges if e.inode in nodes and e.jnode in nodes], key=lambda edge: edge.savings[source_id], reverse=True)

    # Build a dummy solution where a vehicle starts from the source, visits
    # a single node, and then goes to the depot.
    routes = collections.deque()
    for node in nodes:
        # Calculate the distances from the node to the depot and
        # from the source to the node and save them.
        node.from_source = dists[source_id, node.id]
        node.to_depot = dists[node.id, depot_id]
        # Verify if the node can be visited according to the Tmax.
        if node.from_source + node.to_depot > Tmax:
            node.route = None
            continue
        # Eventually construct a new route that goes from the
        # source to the node and from the node to the depot.
        route = Route(source, depot, node)
        node.route = route
        node.link_left = True
        node.link_right = True
        routes.append(route)

    # Merge the routes giving priority to edges with highest efficiency
    for edge in _bra(sorted_edges, beta):
        inode, jnode = edge.inode, edge.jnode
        # If the edge connect nodes already into the same route
        # next edge is considered
        iroute, jroute = inode.route, jnode.route
        if iroute is None or jroute is None or iroute == jroute:
            continue
        # If inode is the last of its route and jnode the first of its
        # route, the merging is possible.
        if inode.link_right and jnode.link_left:
            # Compare the length of the new route to Tmax
            if iroute.cost - inode.to_depot + jroute.cost - jnode.from_source + edge.cost <= Tmax:
                # Merge the routes
                iroute.merge(jroute, edge, dists)
                # Remove the route incorporated into iroute
                routes.remove(jroute)
        # If the number of routes is already equal to the number of vehicles,
        # interrupt the procedure.
        if len(routes) == n_vehicles:
            break

    # Return the solution as a list of the best possible routes.
    return sorted(routes, key=operator.attrgetter("revenue"), reverse=True)[:n_vehicles]


@functools.lru_cache(maxsize=None)
def PJS_cache (problem, source, nodes, depot, alpha):
    """
    Cached implementation of the PJS.
    Used only for heuristic and deterministic behaviour when we do not
    need to explore different solutions.

    :param alpha: The alpha value used to calculate edges savings (used only for caching)

    :return: The routes the vehicles starting from source will make.
    """
    return PJS(problem, source, nodes, depot, beta=0.9999)


def multistartPJS (problem, source, nodes, depot, alpha, maxiter, betarange):
    """
    This method is a multi-start execution of the PJS.
    At each iteration, a new solution is generated by using a different beta
    parameter in the selection of edges for the merging process.

    :param maxiter: The maximum number of iterations.
    :param betarange: The range in which beta is randomly generated at each iteration.
    :return: The best solution found as a set of routes, and the respective revenue.
    """
    # Generate the starting greedy solution
    bestroutes = PJS_cache(problem, source, nodes, depot, alpha)
    bestrevenue = sum(r.revenue for r in bestroutes)

    # Save beta ranges
    betamin, betamax = betarange

    for _ in range(maxiter):

        # Generate a new solution
        routes = PJS(problem, source, nodes, depot, beta=random.uniform(betamin, betamax))
        revenue = sum(r.revenue for r in routes)

        # Eventually update the best
        if revenue > bestrevenue:
            bestroutes, bestrevenue = routes, revenue

    # Return the best solution found so far
    return bestroutes, bestrevenue

In [6]:
def set_savings (problem, alpha=0.3):
    """
    This method calculate the saving of edges according to the given alpha.

    NOTE: Edges are modified in place.

    :param problem: The instance of the problem to solve.
    :param alpha: The alpha parameter of the PJS.
    :return: The problem instance modified in place.
    """
    dists, depot = problem.dists, problem.depot
    for edge in problem.edges:
        cost, inode, jnode = edge.cost, edge.inode, edge.jnode
        revenue = inode.revenue + jnode.revenue
        edge.savings = {
            source.id : (1.0 - alpha)*(dists[inode.id, depot.id] + dists[source.id, jnode.id] - cost) + alpha*revenue
        for source in problem.sources}
    return problem


def alpha_optimisation (problem, alpha_range=np.arange(0.0, 1.1, 0.1)):
    """
    This method is used to optimise the alpha parameter.
    Alpha parameter is used in the calculation of edges savings:

        saving = distance_saving * (1 - alpha) + revenue * alpha

    The higher is alpha the bigger is the importance of the revenue,
    the lower is alpha the bigger is the importance of the distance saving.


    We basically run 10 deterministic executions of the algorithm
    (i.e., Mapper and then PJS) for 10 different levels of alpha.
    The value of alpha that provides the best deterministic solution
    is kept.

    NOTE: This method also changes in place the savings of the edges.

    :param problem: The problem instance to solve .
    :param alpha_range: The levels of alpha to test.
    :return: The best value obtained for alpha.
    """
    # Move useful references to the stack
    dists, depot, sources, nodes = problem.dists, problem.depot, problem.sources, problem.nodes
    # Run once the deterministic mapper
    mapping = mapper(problem, iterator=greedy)

    # Initialise the best alpha to zero
    best_alpha, best_revenue = 0.0, float("-inf")

    # We try different values of alpha parameter and we keep the best
    for alphatest in alpha_range:
        # Tray a new value of alpha
        alphatest = round(alphatest, 1)
        # Compute the edges savings according to the new alpha value
        set_savings(problem, alphatest)
        # Run a deterministic version of the PJS algorithm for each source.
        routes = []
        for source in problem.sources:
            partial_routes = PJS_cache(problem, source, tuple(source.nodes), depot, alphatest)
            routes.extend(partial_routes)
        # Total obtained revenue (i.e., quality of the solution)
        total_revenue = sum(r.revenue for r in routes)
        # Eventually update the alpha
        if total_revenue > best_revenue:
            best_alpha, best_revenue = alphatest, total_revenue
    # Set the savings of the edges by using the best found alpha
    set_savings(problem, best_alpha)
    # Return the best alpha obtained
    return best_alpha


def heuristic (problem, iterator, alpha):
    """
    This is the main executiom of the solver.
    It can be deterministic of stochastic depending on the iterator
    passed as argument.

    :param problem: The problem instance to solve.
    :param iterator: The iterator to be passed to the mapper.
    :param alpha: The alpha value used to calculate edges savings (used only for caching)
    :return: The solution as a set of routes, their total revenue, the mapping represented a matrix.
    """
    # Mapping
    mapping = mapper(problem, iterator)
    # PJS on routes
    routes = []
    for source in problem.sources:
        r = PJS_cache(problem, source, tuple(source.nodes), problem.depot, alpha)
        routes.extend(r)
    # Calculate total revenue
    revenue = sum(r.revenue for r in routes)
    # Return the mapping, the routes and the revenue
    return revenue, mapping, tuple(routes)


def multistart (problem, alpha, maxiter=1000, betarange=(0.1, 0.3)):
    """
    This is the multistart execution of the PJS algorithm.
    At each iteration a new solution is generated by introducing
    slight modifications through a biased randomised approach.
    The new generated solution is compared to the best one and
    eventually replace them (if the revenue obtained is higher).

    :param problem: The problem instance to solve.
    :param alpha: The alpha value used to calculate edges savings (used only for caching)
    :param maxiter: The maximum number of iterations and different
                    mapping tested.
    :param betarange: The range of the beta parameter to use in the biased randomisation.

    :return: The best solution found so far with the respective mapping and revenue.
    """
    # Check the values provided for the beta parameter
    if betarange[0] > betarange[1]:
        raise Exception("Min beta should be higher than max beta.")

    # Save beta ranges
    minbeta, maxbeta = betarange

    # Initialise the starting solution as the greedy one
    brevenue, bmapping, broutes = heuristic(problem, iterator=greedy, alpha=alpha)

    # Iterated Local Search
    for i in range(maxiter):

        # Initialise the biased randomised iterator
        _bra = functools.partial(BRA, beta=random.uniform(minbeta, maxbeta))

        # Generate a new solution
        revenue, mapping, routes = heuristic(problem, iterator=_bra, alpha=alpha)

        # Eventually update the best
        if revenue > brevenue:
            brevenue, bmapping, broutes = revenue, mapping, routes

    # Return the best solution found so far
    return brevenue, bmapping, broutes


def multistart_keep_elites (problem, alpha, maxiter=1000, betarange=(0.1, 0.3), nelites=5):
    """
    Same as the multistart, but instead of saving just the best solution, we keep
    track of the nelites best ones storing them in a heap.

    :param problem: The problem instance to solve.
    :param alpha: The alpha value used to calculate edges savings (used only for caching)
    :param maxiter: The maximum number of iterations and different
                    mapping tested.
    :param betarange: The range of the beta parameter to use in the biased randomisation.
    :param nelites: The number of elite solutions we keep in memory.

    :return: The best solution found so far with the respective mapping and revenue.
    """
    # Check the values provided for the beta parameter
    if betarange[0] > betarange[1]:
        raise Exception("Min beta should be higher than max beta.")

    # Save beta ranges
    minbeta, maxbeta = betarange

    # Initialise the heap of the best solutions
    bestsolutions = []

    # Initialise the count of insered solutions to avoid comparison of not comparable
    # elements into the heap
    count = 0

    # Initialise the starting solution as the greedy one (i.e., brevenue, bmapping, broutes)
    revenue, routes, mapping = heuristic(problem, iterator=greedy, alpha=alpha)

    # Update the heap
    heapq.heappush(bestsolutions, (revenue, count, routes, mapping ))
    count += 1

    # Iterated Local Search
    for i in range(maxiter):

        # Initialise the biased randomised iterator
        _bra = functools.partial(BRA, beta=random.uniform(minbeta, maxbeta))

        # Generate a new solution (i.e., revenue, mapping, routes)
        revenue, routes, mapping = heuristic(problem, iterator=_bra, alpha=alpha)

        # Eventually update the best
        if len(bestsolutions) == nelites and revenue > bestsolutions[0][0]:
            heapq.heappushpop(bestsolutions, (revenue, count, routes, mapping))
            count += 1

        # Update the heap if its capacity is not saturated
        if len(bestsolutions) < nelites:
            heapq.heappush(bestsolutions, (revenue, count, routes, mapping))
            count += 1

    # Return the best solution found so far
    return tuple(bestsolutions)


def optimise_elites (problem, elites, alpha, maxiter=1000, betarange=(0.1, 0.3)):
    """
    This process is used to optimise the elite solutions using a multistart PJS.

    :param problem: The problem instance to solve.
    :param elites: The elite solutions presented as sets of routes.
    :param alpha: The alpha value used to calculate edges savings (used only for caching)
    :param maxiter: The number of solutions explored.
    :param betarange: The range of beta used for the generation of different solutions
                        with a biased randomised approach.
    :return: The best solution chosen among the optimised elites.
    """
    # Initialise the current best
    bestroutes, bestrevenue, bestmapping = elites[0][3], elites[0][0], elites[0][2]

    S = len(problem.sources)

    for _, _, mapping, _ in elites:

        # Init the optimised routes and revenue
        total_routes, total_revenue = [], 0

        # Run a multi start PJS on each group of nodes assigned to a single source
        for i, source in enumerate(problem.sources):

            nodes = tuple(node for node, v in zip(problem.nodes, mapping[i, S:]) if v == 1)

            routes, revenue = multistartPJS(problem, source, nodes, problem.depot, alpha, maxiter, betarange)

            total_routes.extend(routes)
            total_revenue += revenue

        # Eventually update the best
        if total_revenue > bestrevenue:
            bestroutes, bestrevenue, bestmapping = total_routes, total_revenue, mapping

    # Return the best routes, revenue, and mapping
    return bestrevenue, bestmapping, tuple(bestroutes)

In [7]:
import os
import networkx as nx
import matplotlib.pyplot as plt


# Default colors for nodes, source nodes, adn depot
NODES_COLOR = '#FDDD71'
SOURCES_COLORS = ('#8FDDF4', '#8DD631', '#A5A5A5', '#DB35EF', '#8153AB')
DEPOT_COLOR = '#F78181'


def euclidean (inode, jnode):
    """
    The euclidean distance between two nodes.

    :param inode: First node.
    :param jnode: Second node.
    """
    return math.sqrt((inode.x - jnode.x)**2 + (inode.y - jnode.y)**2)


class Problem:
    """
    An instance of this class represents a single-source Team Orienteering
    Problem.

    It may also be translated according to different rules in a multi-source
    version of it.
    """

    def __init__(self, name, n_nodes, n_vehicles, Tmax, sources, nodes, depot):
        """
        Initialise.

        :param name: The name of the problem
        :param n_nodes: The number of nodes.
        :param n_vehicles: The number of vehicles / paths.
        :param Tmax: The maximum distance vehicles can run / time budget for paths.
        :param sources: The source nodes.
        :param nodes: The nodes to visit.
        :param depot: The depot.

        :attr dists: The matrix of distances between nodes.
        :attr positions: A dictionary of nodes positions.
        :attr edges: The edges connecting the nodes.
        """
        self.name = name
        self.n_nodes = n_nodes
        self.n_vehicles = n_vehicles
        self.Tmax = Tmax
        self.sources = sources
        self.nodes = nodes
        self.depot = depot

        # Initialise edges list and nodes positions
        edges = collections.deque()
        dists = np.zeros((n_nodes, n_nodes))
        # Calculate the matrix of distances and instantiate the edges
        # and define nodes colors and positions
        source_id = 0
        for node1, node2 in itertools.permutations(itertools.chain(sources, nodes, (depot,)), 2):
            # Calculate the edge cost
            id1, id2 = node1.id, node2.id
            cost = euclidean(node1, node2)
            # Compile the oriented matrix of distances
            dists[id1, id2] = cost
            # Create the edge
            if not node1.isdepot and not node2.issource:
                edges.append(Edge(node1, node2, cost))

        self.dists = dists
        self.edges = edges


    def __hash__(self):
        return id(self)


    def __repr__(self):
        return f"""
        Problem {self.name}
        ---------------------------------------------
        nodes: {self.n_nodes}
        vehicles: {self.n_vehicles}
        Tmax: {self.Tmax}
        multi-source: {self.multi_source}
        ---------------------------------------------
        """

    @property
    def multi_source (self):
        """ A property that says if the problem is multi-source or not. """
        return len(self.sources) > 1


    def iternodes (self):
        """ A method to iterate over all the nodes of the problem (i.e., sources, customers, depot)"""
        return itertools.chain(self.sources, self.nodes, (self.depot,))


def plot (problem, *, routes=tuple(), mapping=None, figsize=(6,4), title=None):
    """
    This method is used to plot a problem using a graph representation that
    makes it easy-to-read.

    :param figsize: The size of the plot.
    :param title: The title of the plot.
    :param routes: The eventual routes found.
    """
    plt.figure(figsize=figsize)
    if title:
        plt.title(title)

    # Build the graph of nodes
    colors, pos = [], {}
    G = nx.DiGraph()
    source_id = 0

    for node in problem.iternodes():
        # Compile the graph
        pos[node.id] = (node.x, node.y)
        G.add_node(node.id)

        # Define nodes colors
        if node.issource:
            colors.append(SOURCES_COLORS[source_id])
            source_id += 1
        elif node.isdepot:
            colors.append(DEPOT_COLOR)
        else:
            if mapping is None:
                colors.append(NODES_COLOR)
            else:
                for i in range(len(problem.sources)):
                    if mapping[i, node.id] == 1:
                        colors.append(SOURCES_COLORS[i] + "60")
                        break

    # Save the routes
    edges = []
    for r in routes:
        # NOTE: Nodes of the route are supposed to always be in the order in which
        # they are stored inside the deque.
        nodes = tuple(r.nodes)
        edges.extend([(r.source.id, nodes[0].id), (nodes[-1].id, r.depot.id)])
        for n1, n2 in zip(nodes[:-1], nodes[1:]):
            edges.append((n1.id, n2.id))

    nx.draw(G, pos=pos, node_color=colors, edgelist=edges, with_labels=True, node_size=100, font_size=6, font_weight="bold")
    plt.show()


def read_problem (filename, path="./IstanzeBenchmark/"):
    """
    This method is used to read a single-source Team Orienteering Problem
    from a file and returns a standard Problem instance.

    :param filename: The name of the file to read.
    :param path: The path where the file is.
    :return: The problem instance.
    """
    
    with open(path + filename, 'r') as file:
        # Read problem parameters
        n_nodes = 102
        n_vehicles = 1
        Tmax = 70
        # Initialise nodes lists
        sources, nodes, depot = [], [], None
        # Read nodes characteristics
        for i, line in enumerate(file):
            node_info = line.split(',')
            if i == 0:
                # Add a source node
                sources.append(Node(i, float(node_info[0]), float(node_info[1]), int(node_info[2]),
                              issource=True, vehicles=n_vehicles))
            elif i == n_nodes - 1:
                # Add the depot
                depot = Node(i, float(node_info[0]), float(node_info[1]), int(node_info[2]), isdepot=True)
            else:
                # Add a node to visit
                nodes.append(Node(i, float(node_info[0]), float(node_info[1]), int(node_info[2])))

        # Instantiate and return the problem
        return Problem(filename, n_nodes, n_vehicles, Tmax, tuple(sources), tuple(nodes), depot)

In [ ]:
import time


problem = read_problem("Bench102.txt")

alpha = alpha_optimisation(problem)

set_savings(problem, alpha=alpha)


print("alpha = " + str(alpha))


_start = time.time()

revenue, mapping, routes = heuristic(problem, greedy, alpha)

print("Heuristic --> Tempo: " + str(time.time() - _start) + " Punteggio: " + str(revenue))

#plot(problem, mapping=mapping, routes=routes, title="Heuristic")


_start = time.time()

revenue, mapping, routes = multistart(problem, alpha, maxiter=1000, betarange=(0.1, 0.3))

print("Multistart --> Tempo: " + str(time.time() - _start) + " Punteggio: " + str(revenue))

#plot(problem, mapping=mapping, routes=routes, title="Multistart")


_start = time.time()

elite_solutions = multistart_keep_elites(problem, alpha, maxiter=1000, betarange=(0.1, 0.3), nelites=5)

print("Multistart Elites --> Tempo: " + str(time.time() - _start) + " Punteggio: " + str(revenue))

#plot(problem, mapping=mapping, routes=routes, title="Multistart Elites")


"""_start = time.time()

revenue, mapping, routes = optimise_elites(problem, elite_solutions, alpha, maxiter=3000, betarange=(0.1, 0.3))

print("Optimise Elites --> Tempo: " + str(time.time() - _start) + " Punteggio: " + str(revenue))

plot(problem, mapping=mapping, routes=routes, title="Optimise Elites")"""


alpha = 0.1
Heuristic --> Tempo: 0.0009586811065673828 Punteggio: 220


In [ ]:
pos={}
for node in problem.iternodes():
        # Compile the graph
        pos[node.id] = (node.x, node.y)
        
# Save the routes
edges = []
for r in routes:
    # NOTE: Nodes of the route are supposed to always be in the order in which
    # they are stored inside the deque.
    nodes = tuple(r.nodes)
    edges.extend([(r.source.id, nodes[0].id), (nodes[-1].id, r.depot.id)])
    for n1, n2 in zip(nodes[:-1], nodes[1:]):
        edges.append((n1.id, n2.id))

        
sortedpos=dict(sorted(edges))
mytour = []
for i in sortedpos:
    mytour.append([pos[i][0],pos[i][1]])
    
mytour.append([pos[101][0],pos[101][1]])

In [ ]:
#Stampa del percorso per vie REALI 
#(ovviamente la distanza in km sarà leggermente diversa da quella in linea d'aria)
import openrouteservice as ors
import folium
import folium.plugins as plugins

# API Key di Open Route Service
ors_key = '5b3ce3597851110001cf6248435cfcfbcf0c42858fde19dccf6f9c0f'

# Richiesta dei servizi tramite API Key di ORS
# Apro un Client per effettuare le richieste al Server di ORS
client = ors.Client(key=ors_key)

# Traccio il percorso
route = client.directions(coordinates=mytour,
                          profile='foot-walking',
                          format='geojson')

#SERVIREBBE AGGIUNGERE DUE MARKER DI DUE COLORI DIFFERENTI PER SEGNALARE LO STARTING POINT E L'END POINT
#OCCHIO - SULLA MAPPA CI SONO DUE AMBIENTI NON COPERTI DALLA STRADA (CON 7 ORE)
#OH NO, ANCHE CON SEI ORE SE FACCIO VILLA DEI MISTERI PIAZZA ESEDRA OH NOOO (ABBASSARE LO SCORE DI QUESTI DUE - vai vai)
map = folium.Map(location=[40.7502816, 14.4868047], zoom_start = 16)
#for sito in percorso:
    #folium.Marker(location = coordinate[sito], tooltip = sito).add_to(map)

for i in range(len(sortedpos)+1):
    if i == 0: #start point 
        folium.Marker(location = (mytour[i][1],mytour[i][0]), tooltip = i, 
                      icon=plugins.BeautifyIcon(icon="arrow-down", icon_shape="marker",
                                                number=i,
                                                border_color= '#b22222',
                                                background_color='#b22222')).add_to(map)
    elif i == len(sortedpos): #end point
        folium.Marker(location = (mytour[i][1],mytour[i][0]), tooltip = i, 
                      icon=plugins.BeautifyIcon(icon="arrow-down", icon_shape="marker",
                                                number=i,
                                                border_color= '#ffd700',
                                                background_color='#ffd700')).add_to(map)
    else:
        folium.Marker(location = (mytour[i][1],mytour[i][0]), tooltip = i, 
                      icon=plugins.BeautifyIcon(icon="arrow-down", icon_shape="marker",
                                                number=i,
                                                border_color= '#b22222',
                                                background_color='#ffffff')).add_to(map)
    
# Aggiungo il GeoJson alla mappa
folium.GeoJson(route, name=('Itinerario Scavi di Pompei')).add_to(map)
    
# Aggiungo il livello del percorso alla mappa
folium.LayerControl().add_to(map)

print('Distanza percorsa in km: ' + str((route['features'][0]['properties']['summary']['distance'])/1000))

map